In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
import pymysql
import pandas as pd

## DB Connection 

Here we can reuse same connection and "never" close it, in an API you cannot because there can be collisions

In [61]:
connection = pymysql.connect(
    host='82.223.54.117',
    port=3306,
    user='flutter_app',
    password=os.environ["MARIA_DB_PASS"],
    database='rubik_app_db'
)

## Dump data

## Read data

In [4]:
comms_guille = pd.read_csv("./test_comms.csv")

In [21]:
comms_guille.head()

,first_letter,second_letter,commutator
0,R,C,(Can U. RB.) U
1,C,R,U' (BR. Can U'.)
2,R,K,u (HZ.) u'
3,K,R,u (ZH.) u'
4,K,M,R U' M2 U R2 U' M2 U R


In [50]:
pd.read_sql("SELECT * FROM edges_stickers", connection).head()

/tmp/ipykernel_15067/3833963140.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT * FROM edges_stickers", connection).head()


,id,user_name,sticker,letter
0,1,sheriff,UR,Te
1,2,sheriff,UL,Ta
2,3,sheriff,UF,Ɐ
3,4,sheriff,UB,Ka
4,5,sheriff,RU,Kʊ


## Limpiar conmutadores

Quitar la palabra 'Can', quitar puntos, etc

In [88]:
import re

In [92]:
def remove_cancellations(commutator: str) -> str:
    """
    Turns 
        (Can U. RB.) U      into
        U' (RB.) U
    """
    commutator = re.sub(r'\(Can (\w)\.\s*', r"\1' (", commutator)
    commutator = re.sub(r'\(Can (\w)\'\.\s*', r"\1 (", commutator)
    commutator = re.sub(r'\(BR\. Can (\w)\'\.\)', r"(BR.) \1", commutator)    
    commutator = re.sub(r'\(BR\. Can (\w)\.\)', r"(BR.) \1'", commutator)
    
    return commutator

In [91]:
def remove_period(commutator):
    """
    Turns 
        U' (RB.) U   into
        U' (RB) U
    """
    return re.sub(r'\((\w{2})\.\)', r'(\1)', commutator)

In [40]:
fns = [lambda x: x, remove_cancellations, remove_period]

In [41]:
def clean_commmutator(commutator, verbose=False):
    for f in fns:
        commutator = f(commutator)
        
        if verbose:
            print(commutator)

    return commutator

In [96]:
print(clean_commmutator("(Can U. RB.) U"))

U' (RB) U


In [43]:
comms_guille["clean_commutator"] = comms_guille.commutator.apply(clean_commmutator)

In [45]:
comms_guille.head()

,first_letter,second_letter,commutator,clean_commutator
0,R,C,(Can U. RB.) U,U' (RB) U
1,C,R,U' (BR. Can U'.),U' (BR) U
2,R,K,u (HZ.) u',u (HZ) u'
3,K,R,u (ZH.) u',u (ZH) u'
4,K,M,R U' M2 U R2 U' M2 U R,R U' M2 U R2 U' M2 U R


## Generalizar conmutadores

### Edges

De las letras de Guille a sticker: B -> UB

Necesitan previa limpia

In [65]:
letters_guille = pd.read_sql("SELECT sticker, letter FROM edges_stickers WHERE user_name='flygorithm'", connection).set_index("letter").sticker

/tmp/ipykernel_15067/3287416019.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  letters_guille = pd.read_sql("SELECT sticker, letter FROM edges_stickers WHERE user_name='flygorithm'", connection).set_index("letter").sticker


In [87]:
letters_guille.head()

letter
C    UR
N    UL
S    UF
B    UB
K    RU
Name: sticker, dtype: object

In [93]:
def commutator_to_sticker_commutator(commutator):
    """
    Turns 
        U' (RB) U         into
        U' {FR,UB} U	
    """
    if "(" not in commutator:
        return commutator

    pre, rest = commutator.split("(")
    inside, post = rest.split(")")
    first_letter, second_letter = inside

    first_sticker = letters_guille[first_letter]
    second_sticker = letters_guille[second_letter]

    return f"{pre}{{{first_sticker},{second_sticker}}}{post}"

In [98]:
print(commutator_to_sticker_commutator("U' (RB) U"))

U' {FR,UB} U


In [98]:
print(commutator_to_sticker_commutator("U' (RB) U"))

U' {FR,UB} U


In [76]:
comms_guille["commutator_stickers"] = comms_guille.clean_commutator.apply(commutator_to_sticker_commutator)

In [79]:
comms_guille["first_sticker"] = comms_guille.first_letter.apply(lambda l: letters_guille[l])

In [80]:
comms_guille["second_sticker"] = comms_guille.second_letter.apply(lambda l: letters_guille[l])

In [85]:
comms_guille.head(10)

,first_letter,second_letter,commutator,clean_commutator,commutator_stickers,first_sticker,second_sticker
0,R,C,(Can U. RB.) U,U' (RB) U,"U' {FR,UB} U",FR,UR
1,C,R,U' (BR. Can U'.),U' (BR) U,"U' {UB,FR} U",UR,FR
2,R,K,u (HZ.) u',u (HZ) u',"u {LF,FU} u'",FR,RU
3,K,R,u (ZH.) u',u (ZH) u',"u {FU,LF} u'",RU,FR
4,K,M,R U' M2 U R2 U' M2 U R,R U' M2 U R2 U' M2 U R,R U' M2 U R2 U' M2 U R,RU,BR
5,R,R,R' U' M2 U R U' M2 U R',R' U' M2 U R U' M2 U R',R' U' M2 U R U' M2 U R',FR,FR
6,M,W,z' U' R U M U' R' U M' z,z' U' R U M U' R' U M' z,z' U' R U M U' R' U M' z,BR,RB
7,R,R,z' M U' R U M' U' R' U z,z' M U' R U M' U' R' U z,z' M U' R U M' U' R' U z,FR,FR
8,W,R,R U' M2 U R U' M2 U R2,R U' M2 U R U' M2 U R2,R U' M2 U R U' M2 U R2,RB,FR
9,W,R,R2 U' M2 U R' U' M2 U R',R2 U' M2 U R' U' M2 U R',R2 U' M2 U R' U' M2 U R',RB,FR


Ahora ya se pueden dumpear a DDBB

In [103]:
comms_guille

"   first_letter second_letter                commutator          clean_commutator       commutator_stickers first_sticker second_sticker\n0             R             C            (Can U. RB.) U                 U' (RB) U              U' {FR,UB} U            FR             UR\n1             C             R          U' (BR. Can U'.)                 U' (BR) U              U' {UB,FR} U            UR             FR\n2             R             K                u (HZ.) u'                 u (HZ) u'              u {LF,FU} u'            FR             RU\n3             K             R                u (ZH.) u'                 u (ZH) u'              u {FU,LF} u'            RU             FR\n4             K             M    R U' M2 U R2 U' M2 U R    R U' M2 U R2 U' M2 U R    R U' M2 U R2 U' M2 U R            RU             BR\n5             R             R   R' U' M2 U R U' M2 U R'   R' U' M2 U R U' M2 U R'   R' U' M2 U R U' M2 U R'            FR             FR\n6             M             W  z'